In [1]:
!pip install pandas

In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
pip install langchain-groq

  Using cached langchain_groq-0.3.8-py3-none-any.whl.metadata (2.6 kB)
Using cached langchain_groq-0.3.8-py3-none-any.whl (16 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_groq import ChatGroq

In [5]:
!pip install python-dotenv

In [6]:
from dotenv import load_dotenv
load_dotenv() # take environment variables from .env
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [7]:
!pip install langchain langchain-groq

In [8]:
!pip install -U langchain-community

  Using cached langchain_community-0.3.30-py3-none-any.whl.metadata (3.0 kB)
  Using cached aiohttp-3.12.15-cp313-cp313-win_amd64.whl.metadata (7.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.11.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached propcache-0.4.0-cp313-cp313-win_amd64.whl.metadata (13 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata (1.1 kB)
Using cached langchain_community-0.3.30-py3-none-any.whl (2.5 MB)
Using cached aiohttp-3.12.15-cp313-cp313-win_amd64.whl (449 kB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached httpx_

In [9]:
!pip install pypdf

In [10]:
from langchain_groq import ChatGroq  # correct Groq import
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
import pypdf  # if you plan to read PDFs

In [11]:
llm = ChatGroq(
    groq_api_key=os.environ["GROQ_API_KEY"],  # replace with your actual key
    model_name="llama-3.1-8b-instant",
    temperature=0.7         # or "mixtral-8x7b" / other available model
)

In [12]:
RESPONSE_JSON={
    "1":{
        "mcq": "multiple choice question",
        "options":{
            "a": "Choice here",
            "b": "Choice here",
            "c": "Choice here",
            "d": "Choice here",
            },
            "correct": "correct answer",
            },
    "2":{
        "mcq": "multiple choice question",
        "options":{
            "a": "Choice here",
            "b": "Choice here",
            "c": "Choice here",
            "d": "Choice here",
            },
            },
    "3":{
        "mcq": "multiple choice question",
        "options":{
            "a": "Choice here",
            "b": "Choice here",
            "c": "Choice here",
            "d": "Choice here",
            },

    }
}

In [13]:
TEMPLATE="""
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [14]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE 
)

In [15]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\LOQ\AppData\Local\Temp\ipykernel_9296\2669661367.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [16]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words form complexity
if the quiz is not at per with the cognitive and analytical abilities of the students.\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}
Check from an expert English Writer of the above quiz: 
"""

In [17]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject","quiz"], template=TEMPLATE)

In [18]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [19]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)

In [20]:
file_path=r"C:\Users\LOQ\QuizChain\data.txt"

In [21]:
with open(r"C:\Users\LOQ\QuizChain\data.txt", "r", encoding="utf-8") as f:
    TEXT = f.read()

In [22]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "Choice here", "b": "Choice here", "c": "Choice here", "d": "Choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "Choice here", "b": "Choice here", "c": "Choice here", "d": "Choice here"}}, "3": {"mcq": "multiple choice question", "options": {"a": "Choice here", "b": "Choice here", "c": "Choice here", "d": "Choice here"}}}'

In [23]:
NUMBER=5
SUBJECT="machine learning"
TONE="simple"

In [27]:
response = generate_evaluate_chain({
    "text": TEXT,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone": TONE,
    "response_json": json.dumps(RESPONSE_JSON)
    })
print(response)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[6][7] The synonym self-teaching computers was also used in this time period.[8][9]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons u

In [ ]:
# print(f"Total Tokens: {cb.total_tokens}")
# print(f"Prompt Tokens: {cb.prompt_tokens}")
# print(f"Completion Tokens: {cb.completion_tokens}")
# print(f"Total Cost: {cb.total_cost}")

In [28]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[6][7] The synonym self-teaching computers was also used in this time period.[8][9]\n\nThe earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[10] Other researchers who have studied human cognitive sys

In [29]:
quiz=response.get("quiz")

In [30]:
quiz=json.loads(quiz)

In [31]:
quiz_table_data=[]
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct=value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [32]:
quiz=pd.DataFrame(quiz_table_data)